In [ ]:
%load_ext autoreload
%autoreload 2

from __future__ import annotations

import re
import ipywidgets as widgets
from typing import TYPE_CHECKING

from app.gpt import ChatGPTAccessor
from app.prompt import EnglishPracticeGPTPromptGenerator


if TYPE_CHECKING:
    from typing import Iterable


class EnglishPracticeGPTInterface:
    def __init__(self) -> None:
        self.wordlist_textarea = widgets.Textarea(
            description="Vocabulary:", layout=widgets.Layout(height="100px")
        )
        self.n_used_words_slider = widgets.IntSlider(
            value=3, min=1, max=10, step=1, description="Number of used words:"
        )
        self.n_used_words_slider.style.description_width = "50%"
        self.n_output_words_slider = widgets.IntSlider(
            value=50,
            min=30,
            max=100,
            step=10,
            description="Number of output words:"
        )
        self.n_output_words_slider.style.description_width = "50%"
        self.allow_repetition_checkbox = widgets.Checkbox(
            value=False, description="Allow word repetition"
        )

        generate_prompt_button = widgets.Button(description="Generate prompt")
        clear_button = widgets.Button(description="Clear")
        generate_text_button = widgets.Button(description="Generate text")
        generate_prompt_button.on_click(self.on_generate_prompt)
        clear_button.on_click(self.on_clear)
        generate_text_button.on_click(self.on_generate_text)
        buttons_hbox = widgets.HBox(
            [generate_prompt_button, clear_button, generate_text_button]
        )

        self.prompt_textarea = widgets.Textarea(
            description="Prompt:",
            layout=widgets.Layout(width="800px", height="190px")
        )
        self.output_textarea = widgets.Textarea(
            description="TEXT:",
            layout=widgets.Layout(width="1100px", height="200px")
        )

        input_widgets_vbox = widgets.VBox(
            [
                self.wordlist_textarea, self.n_used_words_slider,
                self.n_output_words_slider, self.allow_repetition_checkbox,
                buttons_hbox
            ]
        )

        self.toplevel_container = widgets.HBox(
            [input_widgets_vbox, self.prompt_textarea]
        )
        self.toplevel_container = widgets.VBox(
            [
                widgets.HBox([input_widgets_vbox, self.prompt_textarea]),
                self.output_textarea
            ]
        )

    def on_generate_prompt(self, _) -> None:
        self.prompt_textarea.value = self._generate_prompt()

    def on_clear(self, _) -> None:
        self.wordlist_textarea.value = ""
        self.allow_repetition_checkbox.value = False
        self.output_textarea.value = ""

    def on_generate_text(self, _) -> None:
        prompt = self._generate_prompt()
        gpt_accessor = ChatGPTAccessor()
        raw_text = gpt_accessor.query(prompt)
        english_practice_text = generate_word_fill_in_practice_text(
            raw_text, words=self._get_words()
        )
        self.output_textarea.value = english_practice_text

    def display(self) -> DisplayHandle:
        display(self.toplevel_container)

    def _generate_prompt(self) -> str:
        words = self._get_words()
        prompt_generator = EnglishPracticeGPTPromptGenerator(
            vocabulary=words,
            n_used_words=self.n_used_words_slider.value,
            n_output_words=self.n_output_words_slider.value,
            allow_repetition=self.allow_repetition_checkbox.value
        )
        return prompt_generator.generate()

    def _get_words(self) -> Iterable[str]:
        return [
            word_stripped for word in self.wordlist_textarea.value.split()
            if (word_stripped := word.strip())
        ]


def generate_word_fill_in_practice_text(
    text: str, words: Iterable[str], blank_string: str = "__________"
) -> str:
    replace_pattern = re.compile("|".join(words), re.IGNORECASE)
    text_with_blanks = replace_pattern.sub(blank_string, text)
    text_with_word_hints = text_with_blanks + "\n\n" + "\n".join(
        word.upper() for word in words
    )
    return text_with_word_hints


EnglishPracticeGPTInterface().display()

In [ ]:
import re

vocabulary = ["stick to your guns", "vigilant", "prudence", "open-minded"]

multi_word_patterns = (r"\s+".join(word.strip().split()) for word in vocabulary)
vocabulary_items_patterns = (rf"\s+{pattern}\s*(?![a-zA-Z])" for pattern in multi_word_patterns)
item_replace_patterns = re.compile("|".join(vocabulary_items_patterns), re.IGNORECASE | re.MULTILINE)
text = "You must be vigilant, exercise your prudence and open-mindedness accordingly, but stick to your guns nevertheless."
item_replace_patterns.sub(" ___ ", text)

In [20]:
import re

text = "hello world"
regex = r"(?:^|\s)hello"
matches = re.findall(regex, text)
print(matches) 

['hello']


In [4]:
import re

text = "I'm looking for something, but I'm not sure what it is."

# Define the regular expression pattern
pattern = r"\bsomething\b(?=[\s,.!?])"

# Use re.search to find the first match in the text
match = re.search(pattern, text)

if match:
    print("Match found at index:", match.start(), match.end())
else:
    print("No match found.")

Match found at index: 16 25


In [6]:
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from reportlab.platypus import Paragraph, Spacer
from reportlab.lib.enums import TA_JUSTIFY

# Create a new PDF file
pdf_file = canvas.Canvas('justified_text_with_headline.pdf', pagesize=letter)

# Set the page dimensions and margins
width, height = letter
left_margin = 1*inch
right_margin = 1*inch
top_margin = 1*inch
bottom_margin = 1*inch

# Create a new style sheet
style_sheet = getSampleStyleSheet()

# Set the style for a justified paragraph
justified_style = style_sheet['Normal']
justified_style.alignment = TA_JUSTIFY

# Set the style for a headline
headline_style = style_sheet['Heading1']

# Define the headline text
headline_text = 'Example Headline'

# Create a headline paragraph object with the headline style
headline_paragraph = Paragraph(headline_text, headline_style)

# Add the headline paragraph to the canvas
headline_paragraph.wrapOn(pdf_file, width - left_margin - right_margin, height - top_margin - bottom_margin)
headline_paragraph.drawOn(pdf_file, left_margin, height - top_margin - headline_paragraph.height)

# Define some text
text = """Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla nec vestibulum risus. Quisque eu ultricies ex. Aenean sit amet erat id dolor dictum lobortis. Duis bibendum elit in ipsum eleifend, eu pulvinar libero bibendum. Nulla facilisi. Sed pellentesque quam vel risus dapibus iaculis. Sed fringilla ut augue id ullamcorper. Sed molestie massa sit amet urna facilisis, at tristique mi finibus. Vestibulum quis tellus blandit, efficitur quam non, hendrerit ante. Vivamus ac augue a leo facilisis sagittis id eget arcu. Nullam cursus elit ac quam elementum consequat. In feugiat sapien non enim pretium, non molestie justo sagittis. Sed sit amet lorem nulla. Fusce sed placerat tellus, a posuere tellus. Nam nec mauris et nisl ultricies commodo in eu justo. Nulla tristique tellus a libero commodo, id iaculis arcu semper. Morbi scelerisque, magna sed molestie pellentesque, velit velit interdum mi, eget consequat ante lacus a nisi."""

# Create a paragraph object with the justified style
paragraph = Paragraph(text, justified_style)

# Add the paragraph to the canvas
paragraph.wrapOn(pdf_file, width - left_margin - right_margin, height - top_margin - bottom_margin - headline_paragraph.height - Spacer(0, 0.2*inch).height)
paragraph.drawOn(pdf_file, left_margin, height - top_margin - headline_paragraph.height - Spacer(0, 0.2*inch).height - paragraph.height)

# Save the PDF file
pdf_file.save()


In [9]:
data = {"F":3, "A": 3, "B": 4, "C": 10}
"\n".join(map(str, sorted(data.items())))

"('A', 3)\n('B', 4)\n('C', 10)\n('F', 3)"

In [4]:
%load_ext autoreload
%autoreload 2

from app.output import ConsoleOutputGenerator
from app.exam import VocabularyExamData

text = """Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla nec vestibulum risus. Quisque eu ultricies ex. Aenean sit amet erat id dolor dictum lobortis. Duis bibendum elit in ipsum eleifend, eu pulvinar libero bibendum. Nulla facilisi. Sed pellentesque quam vel risus dapibus iaculis. Sed fringilla ut augue id ullamcorper. Sed molestie massa sit amet urna facilisis, at tristique mi finibus. Vestibulum quis tellus blandit, efficitur quam non, hendrerit ante. Vivamus ac augue a leo facilisis sagittis id eget arcu. Nullam cursus elit ac quam elementum consequat. In feugiat sapien non enim pretium, non molestie justo sagittis. Sed sit amet lorem nulla. Fusce sed placerat tellus, a posuere tellus. Nam nec mauris et nisl ultricies commodo in eu justo. Nulla tristique tellus a libero commodo, id iaculis arcu semper. Morbi scelerisque, magna sed molestie pellentesque, velit velit interdum mi, eget consequat ante lacus a nisi."""
correct_answers = {
    "B": "first",
    "C": "second"
}
exam_data = VocabularyExamData("", text_with_blanks=text, correct_answers=correct_answers)
ConsoleOutputGenerator().generate_output(exam_data)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Vocabulary Practice - Fill In the Blanks

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla nec
vestibulum risus. Quisque eu ultricies ex. Aenean sit amet erat id
dolor dictum lobortis. Duis bibendum elit in ipsum eleifend, eu
pulvinar libero bibendum. Nulla facilisi. Sed pellentesque quam vel
risus dapibus iaculis. Sed fringilla ut augue id ullamcorper. Sed
molestie massa sit amet urna facilisis, at tristique mi finibus.
Vestibulum quis tellus blandit, efficitur quam non, hendrerit ante.
Vivamus ac augue a leo facilisis sagittis id eget arcu. Nullam cursus
elit ac quam elementum consequat. In feugiat sapien non enim pretium,
non molestie justo sagittis. Sed sit amet lorem nulla. Fusce sed
placerat tellus, a posuere tellus. Nam nec mauris et nisl ultricies
commodo in eu justo. Nulla tristique tellus a libero commodo, id
iaculis arcu semper. Morbi scelerisque, magna sed molestie
pellente